In [22]:
import numpy as np
import open3d as o3d
from pykinect2 import PyKinectV2
from pykinect2 import PyKinectRuntime



In [41]:
# Main Settings

# Angle between each scan in degrees
ANGLE_INCR: int = 45

MIN_DEPTH: int = 800
MAX_DEPTH: int = 1500

MIN_X: int = 200
MAX_X: int = 420

MIN_Y: int = 120
MAX_Y: int = 217

# The target object type.
TARGET: str = 'sphere'

In [3]:
# Initialize Kinect
kinect = PyKinectRuntime.PyKinectRuntime(PyKinectV2.FrameSourceTypes_Depth)

In [4]:
# Initialize Pointclouds Container
pcds = []

In [ ]:
#viz = o3d.visualization.Visualizer()
#viz.create_window(window_name='KinectScan', width=720, height=480, visible=True)

In [42]:
for angle in range(0, 360, ANGLE_INCR):
    
    # Wait for Input
    i = input("Press Enter to continue...")
    if i == 'q':
        break

    print(f"Capturing Pointcloud at {angle} degrees.")

    depth_frame = np.reshape(kinect.get_last_depth_frame(), (424, 512)).astype(np.uint16)                       # type: ignore

    # Filter Image
    depth_frame = np.where((depth_frame >= MIN_DEPTH) & (depth_frame <= MAX_DEPTH), depth_frame, 0)
    depth_frame[:MIN_Y, :] = 0
    depth_frame[MAX_Y:, :] = 0
    depth_frame[:, :MIN_X] = 0
    depth_frame[:, MAX_X:] = 0

    # Create Open3D Depth Image from Raw Depth Frame
    depth_image = o3d.geometry.Image(depth_frame)

    # Create Pointcloud
    pcd = o3d.geometry.PointCloud.create_from_depth_image(
        depth_image,
        o3d.camera.PinholeCameraIntrinsic(
            o3d.camera.PinholeCameraIntrinsicParameters.Kinect2DepthCameraDefault
        )
    )

    # Rotate Pointcloud
    print(f'Rotating Pointcloud by {angle} degrees...')
    rotation = o3d.geometry.get_rotation_matrix_from_xyz((0, np.deg2rad(angle), 0))
    pcd.rotate(rotation)

    # Visualize Pointcloud
    if angle == 0:
        print("Visualizing Pointcloud")
        o3d.visualization.draw_geometries([pcd])
        
        if input("Continue? [y/n]").lower() == 'n':
            break

    # Save the Pointcloud
    print("Writing to File")
    o3d.io.write_point_cloud(f'../data/Kinect/{TARGET}/raw/{angle}.ply', pcd)

    pcds.append(pcd)



kinect.close()

Capturing Pointcloud at 0 degrees.
Rotating Pointcloud by 0 degrees...
Visualizing Pointcloud


In [ ]:
del kinect

In [ ]:
# Register Collected Pointclouds

assert len(pcds) > 0, "-- No Pointclouds Collected --"
print(f'-- {len(pcds)} Pointclouds Collected --')


final_pcd = pcds[0]
for pcd in pcds[1:]:
    print("Performing ICP Registration...")
    icpRegResults = o3d.pipelines.registration.registration_icp(
        pcd, 
        final_pcd, 
        0.1, 
        np.identity(4), 
        o3d.pipelines.registration.TransformationEstimationPointToPoint(), 
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1000)
    )

    pcd.transform(icpRegResults.transformation)
    final_pcd += pcd

In [ ]:
# Visualize Result

o3d.visualization.draw_geometries([final_pcd])

In [ ]:
# Save Result

o3d.io.write_point_cloud(f'../data/Kinect/{TARGET}/result.ply', final_pcd)